In [1]:
import pandas as pd
import jsonlines
from typing import List, Optional
import sys
sys.path.append("../")
from src.engine.embedding import Embedding

In [2]:
data = pd.read_parquet("../data/yappy_hackaton_2024_40k_v1_to_elastic.pq")

In [3]:
embedding = Embedding(device="cuda:0")

In [4]:
concat_fields = list("passage: " + data["description_postprocessed"] + " " + data["title_shazam_postprocessed"] + " " + data["subtitle_shazam_postprocessed"])
# concat_fields = concat_fields[:1000]

In [5]:
bs = 500
text_embeddings: List[List[float]] = []
for i in range(0, len(concat_fields), bs):
    batch = concat_fields[i:i+bs]
    text_embeddings.extend(embedding(texts=batch))

In [6]:
data["embedding"] = text_embeddings

In [21]:
def create_documents_jsonl(data: Optional[pd.DataFrame] = None, path_to_pq: Optional[str] = None, path_to_save: str = "../data/documents.jsonl"):
    if data is None and path_to_csv is None:
        raise Exception
    if data is None:
        data = pd.read_parquet(path_to_pq)
        
    with jsonlines.open(path_to_save, mode="a") as writer:
        for idx, row in data.iterrows():
            sample = {}
            sample["doc_id"] = row["index orig"]
            sample["embedding"] = row["embedding"]
            sample["text_hashtags"] = row["description_postprocessed"]
            sample["video_hashtags"] = ""
            sample["audio_transcription"] = row["transcription_postprocessed"]
            sample["song_name"] = row["title_shazam_postprocessed"]
            sample["song_author"] = row["subtitle_shazam_postprocessed"]
            writer.write(sample)